In [142]:
import pandas as pd
df_predict = pd.read_csv('predict.csv')
df_data = pd.read_csv('data.csv')

In [143]:
def prediction_price(df_data, df_predict, city, isluxury):
    df_booking = df_data.copy()
    df_ponderaciones = df_predict.copy()
    df_booking.groupby(['noches','n_personas','city', 'luxury'])['precio'].mean().reset_index()
    df = df_booking.groupby(['noches','n_personas','city', 'luxury'])['precio'].mean().reset_index()
    df_temp = df[(df['city'] == city) & (df['luxury'] == isluxury)]
    precio_1n_1p = df_temp[(df_temp['n_personas'] == 1) & (df_temp['noches'] == 1)]['precio']
    precio_2n_2p = df_temp[(df_temp['n_personas'] == 2) & (df_temp['noches'] == 2)]['precio']
    precio_1n_2p = df_temp[(df_temp['n_personas'] == 2) & (df_temp['noches'] == 1)]['precio']

    df_ponderaciones['Date'] = pd.to_datetime(df_ponderaciones['Date'], format='%Y-%m-%d')
    df_ponderaciones.set_index('Date', inplace=True)
    train_data = df_ponderaciones[df_ponderaciones['city'] == city]['precios_hoteleros']['2023-01-01':'2025-12-01']
    train_data = train_data[~train_data.index.duplicated(keep='first')]
    df =train_data.reset_index()

    precio_ref = df.loc[df['Date'] == '2024-09-01', 'precios_hoteleros'].values[0]

    # Filtrar los precios de 2025
    df_2025 = df[df['Date'].dt.year == 2025].copy()

    # Calcular la variación porcentual
    df_2025['variacion_porcentual'] = ((df_2025['precios_hoteleros'] - precio_ref) / precio_ref)

    # Mostrar el resultado
    df_2025[['Date', 'precios_hoteleros', 'variacion_porcentual']]
    df_2025['precios_1p_1n'] = df_2025['variacion_porcentual'].apply(lambda x : round((x+1)*precio_1n_1p, 2))
    df_2025['precios_2p_1n'] = df_2025['variacion_porcentual'].apply(lambda x : round((x+1)*precio_1n_2p, 2))
    df_2025['precios_2p_2n'] = df_2025['variacion_porcentual'].apply(lambda x : round((x+1)*precio_2n_2p, 2))
    return df_2025[['Date','precios_1p_1n','precios_2p_1n','precios_2p_2n']]

In [144]:
final = pd.DataFrame(columns=['Date','city','precios_1p_1n','precios_2p_1n','precios_2p_2n','luxury'])

for i in range(1,18,1):
    for x in [1,0]:
        temp_df = prediction_price(df_data, df_predict, i, x)
        if x == 0:
            temp_df['luxury'] = 'No Lujo'
        else:
            temp_df['luxury'] = 'Lujo'
        temp_df['city'] = i
        final = pd.concat([final,temp_df], axis=0, ignore_index=True)

final

C:\Users\FX516\AppData\Local\Temp\ipykernel_17544\2814789514.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final = pd.concat([final,temp_df], axis=0, ignore_index=True)


,Date,city,precios_1p_1n,precios_2p_1n,precios_2p_2n,luxury
0,2025-01-01,1,320.65,340.83,534.30,Lujo
1,2025-02-01,1,330.77,351.58,551.17,Lujo
2,2025-03-01,1,322.40,342.69,537.22,Lujo
3,2025-04-01,1,352.44,374.62,587.28,Lujo
4,2025-05-01,1,434.92,462.28,724.71,Lujo
...,...,...,...,...,...,...
403,2025-08-01,17,119.59,124.19,286.19,No Lujo
404,2025-09-01,17,119.68,124.27,286.40,No Lujo
405,2025-10-01,17,122.46,127.16,293.05,No Lujo
406,2025-11-01,17,114.43,118.82,273.83,No Lujo


In [145]:
final.to_csv('predicciones.csv', index=False)